# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 27 2022 12:54PM,238279,15,PNC359933B,"Person & Covey, Inc.",Released
1,May 27 2022 12:54PM,238279,15,PNC362838A,"Person & Covey, Inc.",Released
2,May 27 2022 12:54PM,238279,15,PNC362916A,"Person & Covey, Inc.",Released
3,May 27 2022 12:54PM,238279,15,PNC362927A,"Person & Covey, Inc.",Released
4,May 27 2022 12:54PM,238279,15,PNC362929A,"Person & Covey, Inc.",Released
5,May 27 2022 12:49PM,238284,10,LOA158415,Beach Products Inc,Released
6,May 27 2022 12:46PM,238283,16,CD00000335,Sartorius Stedim North America,Released
7,May 27 2022 12:42PM,238282,16,SAR00000335,Sartorius Stedim North America,Released
8,May 27 2022 12:39PM,238281,10,8069319,"Snap Medical Industries, LLC",Released
9,May 27 2022 12:39PM,238281,10,8069320,"Snap Medical Industries, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,238279,Released,5
31,238281,Released,3
32,238282,Released,1
33,238283,Released,1
34,238284,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
238279,NaN,NaN,NaN,5.0
238281,NaN,NaN,NaN,3.0
238282,NaN,NaN,NaN,1.0
238283,NaN,NaN,NaN,1.0
238284,NaN,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
238124,0.0,0.0,0.0,1.0
238179,0.0,0.0,0.0,1.0
238181,0.0,0.0,1.0,0.0
238191,0.0,0.0,0.0,1.0
238200,1.0,0.0,0.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
238124,0.0,0,0,1
238179,0.0,0,0,1
238181,0.0,0,1,0
238191,0.0,0,0,1
238200,1.0,0,0,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,238124,0.0,0,0,1
1,238179,0.0,0,0,1
2,238181,0.0,0,1,0
3,238191,0.0,0,0,1
4,238200,1.0,0,0,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,238124,0.0,,,1
1,238179,0.0,,,1
2,238181,0.0,,1,
3,238191,0.0,,,1
4,238200,1.0,,,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 27 2022 12:54PM,238279,15,"Person & Covey, Inc."
5,May 27 2022 12:49PM,238284,10,Beach Products Inc
6,May 27 2022 12:46PM,238283,16,Sartorius Stedim North America
7,May 27 2022 12:42PM,238282,16,Sartorius Stedim North America
8,May 27 2022 12:39PM,238281,10,"Snap Medical Industries, LLC"
11,May 27 2022 12:20PM,238124,16,Sartorius Stedim North America
12,May 27 2022 12:20PM,238278,20,"ACI Healthcare USA, Inc."
44,May 27 2022 12:00PM,238275,10,Beach Products Inc
45,May 27 2022 11:54AM,238274,102,"Senseonics, Incorporated"
46,May 27 2022 11:53AM,238273,19,"Emersa Waterbox, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Completed,Executing,Released
0,May 27 2022 12:54PM,238279,15,"Person & Covey, Inc.",0.0,,,5
1,May 27 2022 12:49PM,238284,10,Beach Products Inc,0.0,,,1
2,May 27 2022 12:46PM,238283,16,Sartorius Stedim North America,0.0,,,1
3,May 27 2022 12:42PM,238282,16,Sartorius Stedim North America,0.0,,,1
4,May 27 2022 12:39PM,238281,10,"Snap Medical Industries, LLC",0.0,,,3
5,May 27 2022 12:20PM,238124,16,Sartorius Stedim North America,0.0,,,1
6,May 27 2022 12:20PM,238278,20,"ACI Healthcare USA, Inc.",0.0,,,32
7,May 27 2022 12:00PM,238275,10,Beach Products Inc,0.0,,,1
8,May 27 2022 11:54AM,238274,102,"Senseonics, Incorporated",0.0,,,1
9,May 27 2022 11:53AM,238273,19,"Emersa Waterbox, LLC",0.0,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 27 2022 12:54PM,238279,15,"Person & Covey, Inc.",5,,
1,May 27 2022 12:49PM,238284,10,Beach Products Inc,1,,
2,May 27 2022 12:46PM,238283,16,Sartorius Stedim North America,1,,
3,May 27 2022 12:42PM,238282,16,Sartorius Stedim North America,1,,
4,May 27 2022 12:39PM,238281,10,"Snap Medical Industries, LLC",3,,
5,May 27 2022 12:20PM,238124,16,Sartorius Stedim North America,1,,
6,May 27 2022 12:20PM,238278,20,"ACI Healthcare USA, Inc.",32,,
7,May 27 2022 12:00PM,238275,10,Beach Products Inc,1,,
8,May 27 2022 11:54AM,238274,102,"Senseonics, Incorporated",1,,
9,May 27 2022 11:53AM,238273,19,"Emersa Waterbox, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 27 2022 12:54PM,238279,15,"Person & Covey, Inc.",5,,
1,May 27 2022 12:49PM,238284,10,Beach Products Inc,1,,
2,May 27 2022 12:46PM,238283,16,Sartorius Stedim North America,1,,
3,May 27 2022 12:42PM,238282,16,Sartorius Stedim North America,1,,
4,May 27 2022 12:39PM,238281,10,"Snap Medical Industries, LLC",3,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 27 2022 12:54PM,238279,15,"Person & Covey, Inc.",5.0,NaN,NaN
1,May 27 2022 12:49PM,238284,10,Beach Products Inc,1.0,NaN,NaN
2,May 27 2022 12:46PM,238283,16,Sartorius Stedim North America,1.0,NaN,NaN
3,May 27 2022 12:42PM,238282,16,Sartorius Stedim North America,1.0,NaN,NaN
4,May 27 2022 12:39PM,238281,10,"Snap Medical Industries, LLC",3.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 27 2022 12:54PM,238279,15,"Person & Covey, Inc.",5.0,0.0,0.0
1,May 27 2022 12:49PM,238284,10,Beach Products Inc,1.0,0.0,0.0
2,May 27 2022 12:46PM,238283,16,Sartorius Stedim North America,1.0,0.0,0.0
3,May 27 2022 12:42PM,238282,16,Sartorius Stedim North America,1.0,0.0,0.0
4,May 27 2022 12:39PM,238281,10,"Snap Medical Industries, LLC",3.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3335470,89.0,11.0,14.0
102,238274,1.0,0.0,0.0
15,476518,7.0,0.0,0.0
16,1667514,6.0,8.0,0.0
17,238256,1.0,0.0,0.0
19,238273,1.0,0.0,0.0
20,714782,46.0,16.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3335470,89.0,11.0,14.0
1,102,238274,1.0,0.0,0.0
2,15,476518,7.0,0.0,0.0
3,16,1667514,6.0,8.0,0.0
4,17,238256,1.0,0.0,0.0
5,19,238273,1.0,0.0,0.0
6,20,714782,46.0,16.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,89.0,11.0,14.0
1,102,1.0,0.0,0.0
2,15,7.0,0.0,0.0
3,16,6.0,8.0,0.0
4,17,1.0,0.0,0.0
5,19,1.0,0.0,0.0
6,20,46.0,16.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,89.0
1,102,Released,1.0
2,15,Released,7.0
3,16,Released,6.0
4,17,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,15,16,17,19,20
Status,,,,,,,
Completed,14.0,0.0,0.0,0.0,0.0,0.0,1.0
Executing,11.0,0.0,0.0,8.0,0.0,0.0,16.0
Released,89.0,1.0,7.0,6.0,1.0,1.0,46.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,15,16,17,19,20
0,Completed,14.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Executing,11.0,0.0,0.0,8.0,0.0,0.0,16.0
2,Released,89.0,1.0,7.0,6.0,1.0,1.0,46.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,15,16,17,19,20
0,Completed,14.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Executing,11.0,0.0,0.0,8.0,0.0,0.0,16.0
2,Released,89.0,1.0,7.0,6.0,1.0,1.0,46.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()